In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the dataset

In [2]:
df = pd.read_csv('CapstoneDataSet.csv')

In [3]:
df

,Cust_Id,Rating,Movie_Id,Genre,MovieName
0,1488844,3,1,Action,Dinosaur Planet
1,822109,5,1,Action,Dinosaur Planet
2,885013,4,1,Action,Dinosaur Planet
3,30878,4,1,Action,Dinosaur Planet
4,823519,3,1,Action,Dinosaur Planet
...,...,...,...,...,...
1048569,196625,4,241,Horror,North by NorthWest
1048570,138835,3,241,Horror,North by NorthWest
1048571,1400154,5,241,Horror,North by NorthWest
1048572,1018992,3,241,Horror,North by NorthWest


# EDA

In [4]:
missData = df.isnull().sum()
missData

Cust_Id         0
Rating          0
Movie_Id        0
Genre        1086
MovieName       0
dtype: int64

In [5]:
# find out the percentage of missing data
percentMiss = missData[3]/df.shape[0] * 100
print(f'the percentage of missing data = {percentMiss}')

the percentage of missing data = 0.10356922830434476


The percentage of missinng data is less therefore, dropping the nan values 

In [6]:
df.dropna(inplace=True)
df.shape

(1047488, 5)

## Find out the list of most popular and liked genre

In [7]:
genre = df.groupby('Genre')['Genre'].agg('count')
genre.sort_values(ascending=False,inplace=True)

In [8]:
print(f'The most popular genre is : {genre.index[0]}')

The most popular genre is : Historical


In [9]:
genre.index[0]

'Historical'

In [10]:
genreLiked = df.groupby('Genre')['Rating'].mean()

In [11]:
genreLiked.sort_values(ascending=False,inplace=True)

In [12]:
print(f'The most liked genre is : {genreLiked.index[0]}')

The most liked genre is : Horror


# Create Model that finds the best suited Movie for one user in every genre.

In [13]:
# conda install -c conda-forge scikit-surprise

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): conda.anaconda.org:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): conda.anaconda.org:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/msys2/win-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/

,Movie_Id,Genre,MovieName
2,3,Horror,Character
239,241,Horror,North by NorthWest


In [38]:
# Import required libraries
import math

import matplotlib.pyplot as plt

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0517  1.0554  1.0604  1.0558  0.0036  
MAE (testset)     0.8519  0.8570  0.8382  0.8490  0.0080  
Fit time          0.58    0.68    0.74    0.67    0.07    
Test time         0.13    0.19    0.13    0.15    0.03    


{'test_rmse': array([1.05168182, 1.05535253, 1.06044369]),
 'test_mae': array([0.85189938, 0.8570137 , 0.83818345]),
 'fit_time': (0.5809102058410645, 0.6790745258331299, 0.7438008785247803),
 'test_time': (0.12523269653320312, 0.19163155555725098, 0.13127827644348145)}

### Training to get the best movie for each Genre

newDF = df[['Movie_Id','Genre','MovieName']].drop_duplicates()
newDF.reset_index(inplace=True)
newDF.drop(columns=['index'],inplace=True)
newDF['Movie_Id'].nunique()

In [69]:
newDF = df[['Movie_Id','Genre','MovieName']].drop_duplicates()
newDF.reset_index(inplace=True)
newDF.drop(columns=['index'],inplace=True)
newDF = newDF[(newDF['Genre']=='Animation')]
newDF

,Movie_Id,Genre,MovieName
7,8,Animation,What the #$*! Do We Know!?
9,10,Animation,Fighter
11,12,Animation,My Favorite Brunette
13,14,Animation,Nature: Antarctica
15,16,Animation,Screamers
17,18,Animation,Immortal Beloved
19,20,Animation,Seeta Aur Geeta
21,22,Animation,Chump Change
35,36,Animation,Lady Chatterley
56,57,Animation,Richard III


In [76]:
# Training it to one genre - Trial
data = Dataset.load_from_df(df[['Genre','Movie_Id','Rating']],reader)
trainset = data.build_full_trainset()
svd.fit(trainset)
newDF['Estimation_Score']=newDF['Movie_Id'].apply(lambda x: svd.predict('Horror',x).est)
newDF.sort_values('Estimation_Score',ascending=False,inplace=True)
print(newDF.head(5))

     Movie_Id      Genre                             MovieName  \
208       209  Animation  Star Trek: Deep Space Nine: Season 5   
166       167  Animation                            The Chorus   
222       223  Animation            Chappelle's Show: Season 1   
96         97  Animation                         Mostly Martha   
56         57  Animation                           Richard III   

     Estimation_Score  
208          4.280663  
166          4.261265  
222          4.211734  
96           4.027227  
56           4.021617  


In [77]:
newDF.head(20)

,Movie_Id,Genre,MovieName,Estimation_Score
208,209,Animation,Star Trek: Deep Space Nine: Season 5,4.280663
166,167,Animation,The Chorus,4.261265
222,223,Animation,Chappelle's Show: Season 1,4.211734
96,97,Animation,Mostly Martha,4.027227
56,57,Animation,Richard III,4.021617
70,71,Animation,Maya Lin: A Strong Clear Vision,4.015373
17,18,Animation,Immortal Beloved,3.963838
11,12,Animation,My Favorite Brunette,3.799476
124,125,Animation,Nighthawks,3.789832
194,195,Animation,Chasing Freedom,3.737282


In [142]:
bestMovie = pd.DataFrame(columns=['Genre','Movies','Ratings'])
bestMovie['Genre'] = df['Genre'].unique()
worstMovie = pd.DataFrame(columns=['Genre','Movies','Ratings'])
worstMovie['Genre'] = df['Genre'].unique()
for i in range(len(df['Genre'].unique())):
    globals()[f'Genre{i}'] = df[['Movie_Id','Genre','MovieName']].drop_duplicates()
    globals()[f'Genre{i}'].reset_index(inplace=True)
    globals()[f'Genre{i}'].drop(columns=['index'],inplace=True)
    globals()[f'Genre{i}'] = globals()[f'Genre{i}'][(globals()[f'Genre{i}']['Genre']==df['Genre'].unique()[i])]

    data = Dataset.load_from_df(df[['Genre','Movie_Id','Rating']],reader)
    trainset = data.build_full_trainset()
    svd.fit(trainset)
    globals()[f'Genre{i}']['Estimation_Score']=globals()[f'Genre{i}']['Movie_Id'].apply(lambda x: svd.predict(movieGenre[i],x).est)
    globals()[f'Genre{i}'].sort_values('Estimation_Score',ascending=False,inplace=True)
    globals()[f'Genre{i}'].reset_index(inplace=True)
    bestMovie.Movies[i] = globals()[f'Genre{i}'].MovieName[0]
    bestMovie.Ratings[i] = globals()[f'Genre{i}'].Estimation_Score[0]
    worstMovie.Movies[i] = globals()[f'Genre{i}'].MovieName.iloc[-1]
    worstMovie.Ratings[i] =  globals()[f'Genre{i}'].Estimation_Score.iloc[-1]

In [144]:
bestMovie

,Genre,Movies,Ratings
0,Action,Dinosaur Planet,3.805749
1,Comedy,Isle of Man TT 2004 Review,3.501404
2,Horror,North by NorthWest,4.114222
3,Romance,Paula Abdul's Get Up & Dance,2.700786
4,Thriller,Lord of the Rings: The Return of the King: Ext...,4.421561
5,Fiction,Sick,3.081482
6,Animation,Star Trek: Deep Space Nine: Season 5,4.325423
7,Gang,Star Trek: Voyager: Season 1,3.788643
8,Other,Elfen Lied,4.111881
9,Mystery,Davy Crockett: 50th Anniversary Double Feature,3.928817


In [145]:
print('The best movies for different genres are:')
for i in range(len(df['Genre'].unique())):
    print(f'{i}. {bestMovie.Genre[i]}: {bestMovie.Movies[i]} - with ratings of {bestMovie.Ratings[i]}')

The best movies for different genres are:
0. Action: Dinosaur Planet - with ratings of 3.8057493878555593
1. Comedy: Isle of Man TT 2004 Review - with ratings of 3.5014044087618936
2. Horror: North by NorthWest - with ratings of 4.114222043499178
3. Romance: Paula Abdul's Get Up & Dance - with ratings of 2.7007864876103946
4. Thriller: Lord of the Rings: The Return of the King: Extended Edition: Bonus Material - with ratings of 4.421561039756829
5. Fiction: Sick - with ratings of 3.0814817920211284
6. Animation: Star Trek: Deep Space Nine: Season 5 - with ratings of 4.325423045153874
7. Gang: Star Trek: Voyager: Season 1 - with ratings of 3.7886434238842526
8. Other: Elfen Lied - with ratings of 4.111881175290403
9. Mystery: Davy Crockett: 50th Anniversary Double Feature - with ratings of 3.9288169155933077
10. Drama: Woman of the Year - with ratings of 4.009842790238052
11. Crime: Record of Lodoss War: Chronicles of the Heroic Knight - with ratings of 4.015348016781885
12. Sci-Fi: I L

In [146]:
print('The worst movies for different genres are:')
for i in range(len(df['Genre'].unique())):
    print(f'{i}. {worstMovie.Genre[i]}: {worstMovie.Movies[i]} - with ratings of {worstMovie.Ratings[i]}')

The worst movies for different genres are:
0. Action: Dinosaur Planet - with ratings of 3.8057493878555593
1. Comedy: Isle of Man TT 2004 Review - with ratings of 3.5014044087618936
2. Horror: Character - with ratings of 3.785172581323503
3. Romance: Paula Abdul's Get Up & Dance - with ratings of 2.7007864876103946
4. Thriller: 8 Man - with ratings of 2.0750735995007754
5. Fiction: Sick - with ratings of 3.0814817920211284
6. Animation: Chump Change - with ratings of 2.6072190079290003
7. Gang: Cold Blooded - with ratings of 2.3439523844655663
8. Other: Lost in the Wild - with ratings of 2.519685856322965
9. Mystery: Iron Monkey 2 - with ratings of 2.0625927745333827
10. Drama: The Libertine - with ratings of 2.372137053405507
11. Crime: Darkwolf - with ratings of 2.432999562165877
12. Sci-Fi: Horror Vision - with ratings of 1.752408383597728
13. Biography: Searching for Paradise - with ratings of 2.292979414873338
14. War: Cannibal Women in the Avocado Jungle of Death - with ratings o

## To find Best and worst rated Movie

In [154]:
worstMovie.sort_values(by='Ratings', ascending=True, inplace=True)
bestMovie.sort_values(by='Ratings', ascending=False, inplace=True)
print(f"The best movie is {bestMovie.Movies.iloc[0]} has a rating of {bestMovie.Ratings.iloc[0]}\nThe worst movie is {worstMovie.Movies.iloc[0]} has a rating of {worstMovie.Ratings.iloc[0]}")

The best movie is Lord of the Rings: The Return of the King: Extended Edition: Bonus Material has a rating of 4.421561039756829
The worst movie is The Bonesetter has a rating of 1.7468044819103379
